# The Restaurant Decision

Problem: Decide whether to wait for a table at a restaurant,
based on the following attributes:

- Choice: is there an alternative restaurant nearby?
- Bar: is there a comfortable bar area to wait in?
- Day: is today Friday or Saturday?
- Hungry: are we hungry?
- Patron: how many people are in the restaurant?
- Price: what’s the price range?
- Rain: is it raining outside?
- Booking: have we made a reservation?
- Type: what kind of restaurant is it?
- Time: what’s the estimated waiting time?

Build the co-occurrence matrix.
Calculate support, confidence, completeness, lift, and leverage for the following rules. Build the co-occurrence matrix.

In [1]:
import pandas as pd

pd.set_option('display.max_colwidth', None)
restaurant = pd.read_csv('../Data/restaurant.csv.zst')
display(restaurant)

,choice,bar,day,hungry,patron,price,rain,booking,type,time,wait
0,T,F,F,T,some,$$$,F,T,french,0,yes
1,T,F,F,T,full,$,F,F,thai,40,no
2,T,T,F,F,some,$,F,F,swiss,0,yes
3,T,F,T,T,full,$,F,F,thai,20,yes
4,T,F,T,F,full,$$$,F,T,french,60,no
5,F,T,F,T,some,$$,T,F,italian,0,yes
6,F,T,F,F,none,$,T,F,swiss,20,no
7,F,F,F,T,some,$$,T,T,thai,0,yes
8,F,T,T,F,full,$,T,F,swiss,60,no
9,T,T,T,T,full,$$$,F,T,italian,20,no


In [2]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

x_train = restaurant[['choice', 'bar', 'day', 'hungry', 'patron', 'rain', 'booking', 'type']]
le = preprocessing.LabelEncoder()
x_train = pd.DataFrame(columns=x_train.columns, data=le.fit_transform(x_train.values.flatten()).reshape(x_train.shape))
x_train['time'] = restaurant['time']

y = le.fit_transform(restaurant['wait'])

Xd_train, Xd_test, y_train, y_test = train_test_split(x_train, y, test_size=0.35)

display(Xd_train)
print(f'y_train = {y_train}\n')

display(Xd_test)
print(f'y_test = {y_test}\n')

,choice,bar,day,hungry,patron,rain,booking,type,time
10,0,0,0,0,5,0,0,8,0
1,1,0,0,1,3,0,0,8,40
0,1,0,0,1,6,0,1,2,0
7,0,0,0,1,6,1,1,8,0
4,1,0,1,0,3,0,1,2,60
11,1,1,1,1,3,0,0,7,40
9,1,1,1,1,3,0,1,4,20


y_train = [0 0 1 1 0 1 0]



,choice,bar,day,hungry,patron,rain,booking,type,time
8,0,1,1,0,3,1,0,7,60
3,1,0,1,1,3,0,0,8,20
5,0,1,0,1,6,1,0,4,0
6,0,1,0,0,5,1,0,7,20
2,1,1,0,0,6,0,0,7,0


y_test = [0 1 1 0 1]

